
## Tema 2: Introducción a las Redes Neuronales

### Resumen
En este notebook vamos a ver como definir y entrenar una red neuronal. Los pasos a seguir generalmente son:
- Definir la arquitectura de la red neuronal.
- Definir la función loss, métrica y optimizador que vamos a usar durante el entrenamiento.
- Compilar el modelo, i.e, asignarle al modelo las funciones necesarios para su entrenamiento.
- Entrenar el modelo.

Importamos los paquetes necesarios

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers, Input, regularizers
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.metrics import AUC
from tensorflow.keras.losses import SparseCategoricalCrossentropy, MSE, MAE
import PyQt5
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

Para el primer ejemplo vamos a usar el famoso dataset MNIST, el cual se compone por 70.000 imágenes de números escritos a mano de 28x28 píxeles. Veamos un ejemplo:

In [ ]:
%matplotlib inline
(X_train, y_train), (X_validation, y_validation) = mnist.load_data()

print('Número de elementos en el dataset = ',X_train.shape[0]+X_validation.shape[0])
plt.title('Número '+str(y_train[5]))
plt.imshow(X_train[5], cmap='Greys')
plt.show()

Vamos a definir una red neuronal. La red tendrá las siguientes características:
- Una entrada de 28x28, la cual se vectoriza con la capa Flatten, ya que la capa Dense espera un tensor (¿?) de dimensión (batch_size, n_features).
- Una capa oculta con 20 neuronas y función de activación sigmoid.
- La capa output tiene una salida de tamaño 10 (0,1,2...,9) y función de activación softmax(¿?).
- En total la Red Neuronal tiene 15.910 parámetros (¿Por que?).

### Simple model

In [3]:
# capas de la red
input = Input(shape=(28,28))
input_flat = layers.Flatten()(input)
layer_1 = layers.Dense(units=20, activation='sigmoid')(input_flat)
output = layers.Dense(units=10, activation='softmax')(layer_1)

# creamos el modelo
simple_model = Model(inputs=input, outputs=output, name='simple_model')
print(simple_model.summary())

Model: "simple_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 20)                15700     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                210       
Total params: 15,910
Trainable params: 15,910
Non-trainable params: 0
_________________________________________________________________
None


Vamos a definir las funciones necesarias para el entrenamiento y asignárselas a nuestro modelo. Vamos a utilizar el SGD con minibatch para hacer un entrenamiento eficiente. Al ser un problema de clasificación multiclase utilizaremos la función loss SparseCategoricalCrossentropy y la métrica accuracy.

In [4]:
# optimizador
optimizer = SGD(learning_rate=0.01)

# función loss
loss = SparseCategoricalCrossentropy(from_logits=False)

# métrica
metrics = ['accuracy']

# compilamos el modelo
simple_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

Para ver diferentes capas, métricas, losses, optimizadores... podemos visitar las siguientes páginas:
- [Keras API](https://keras.io/api/)
- [TensorFlow](https://www.tensorflow.org/api_docs/python/tf/keras)

En ellas se puede encontrar todo lo necesario para definir y entrenar modelos de deep learning.
Finalmente vamos a entrenar la red neuronal en el dataset y ver que score tiene en el dataset de validación.
El batch_size es el número de elementos que se usa para modificar los pesos de la red en cada paso del entrenamiento. Y las epochs son el número de veces que repetiremos el dataset para entrenar el modelo.

In [5]:
# entrenamos
history = simple_model.fit(x=X_train, y=y_train, batch_size=20, epochs=10,
                 validation_data=(X_validation, y_validation))

Epoch 1/10
3000/3000 [==============================] - 2s 572us/step - loss: 1.2435 - accuracy: 0.6865 - val_loss: 0.8001 - val_accuracy: 0.8244
Epoch 2/10
3000/3000 [==============================] - 2s 510us/step - loss: 0.7394 - accuracy: 0.8085 - val_loss: 0.6554 - val_accuracy: 0.8272
Epoch 3/10
3000/3000 [==============================] - 2s 516us/step - loss: 0.6708 - accuracy: 0.8141 - val_loss: 0.6040 - val_accuracy: 0.8343
Epoch 4/10
3000/3000 [==============================] - 2s 522us/step - loss: 0.5909 - accuracy: 0.8292 - val_loss: 0.5277 - val_accuracy: 0.8533
Epoch 5/10
3000/3000 [==============================] - 2s 516us/step - loss: 0.5444 - accuracy: 0.8408 - val_loss: 0.5005 - val_accuracy: 0.8531
Epoch 6/10
3000/3000 [==============================] - 2s 511us/step - loss: 0.5410 - accuracy: 0.8388 - val_loss: 0.4658 - val_accuracy: 0.8645
Epoch 7/10
3000/3000 [==============================] - 2s 522us/step - loss: 0.5096 - accuracy: 0.8513 - val_loss: 0.4884 -

In [7]:
# predecimos con el modelo entrenado los 10 primero elementos del dataset
y_true = y_validation[:10]
print('y_true:', y_true)

y_pred = simple_model.predict(X_validation[:10])
print('y_pred:', np.argmax(y_pred, axis=-1))  # ¿argmax?

y_true: [7 2 1 0 4 1 4 9 5 9]
y_pred: [7 2 1 0 4 1 4 9 6 9]


In [8]:
y_pred

array([[2.2506874e-04, 1.4558205e-04, 2.4167097e-03, 4.4191037e-03,
        5.1281543e-04, 2.4482154e-04, 5.4255273e-05, 9.7932029e-01,
        1.8867136e-04, 1.2472747e-02],
       [2.6901497e-03, 7.3232348e-03, 6.8590772e-01, 1.5693395e-01,
        2.4564797e-04, 2.0325733e-02, 6.3607641e-02, 3.8052196e-05,
        6.2790297e-02, 1.3780233e-04],
       [2.9783284e-06, 9.2777413e-01, 6.4427435e-02, 1.1191961e-03,
        8.6712709e-04, 1.6972222e-04, 8.1403990e-04, 9.6266728e-04,
        3.7258929e-03, 1.3687951e-04],
       [9.7813797e-01, 1.7601857e-05, 6.9177938e-03, 2.6010459e-03,
        7.4160380e-05, 6.7823417e-03, 2.2694138e-03, 5.6715246e-04,
        2.2544400e-03, 3.7805008e-04],
       [4.1319625e-05, 1.0858639e-03, 1.6002225e-03, 8.4359424e-05,
        8.1083119e-01, 4.8996374e-04, 1.2654632e-03, 2.3539601e-02,
        4.9797841e-04, 1.6056402e-01],
       [2.9783284e-06, 9.2777413e-01, 6.4427435e-02, 1.1191961e-03,
        8.6712709e-04, 1.6972222e-04, 8.1403990e-04, 9.62

### Big model
Podemos generar un modelo un poco más grande para obtener unos mejores resultados.

In [9]:
# capas del modelo
input = Input(shape=(28,28))
input_flat = layers.Flatten()(input)
layer_1 = layers.Dense(units=100, activation='sigmoid')(input_flat)
layer_2 = layers.Dense(units=50, activation='sigmoid')(layer_1)
output = layers.Dense(units=10, activation='softmax')(layer_2)

# creamos y compilamos el modelo
big_model = Model(inputs=input, outputs=output, name='simple_model')
print(big_model.summary())
big_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

Model: "simple_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               78500     
_________________________________________________________________
dense_3 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_4 (Dense)              (None, 10)                510       
Total params: 84,060
Trainable params: 84,060
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
# entrenamos
history = big_model.fit(x=X_train, y=y_train, batch_size=20, epochs=10,
                           validation_data=(X_validation, y_validation))

Epoch 1/10
3000/3000 [==============================] - 2s 718us/step - loss: 1.1523 - accuracy: 0.7594 - val_loss: 0.5888 - val_accuracy: 0.8799
Epoch 2/10
3000/3000 [==============================] - 2s 681us/step - loss: 0.4866 - accuracy: 0.8891 - val_loss: 0.3981 - val_accuracy: 0.9043
Epoch 3/10
3000/3000 [==============================] - 2s 680us/step - loss: 0.3821 - accuracy: 0.9035 - val_loss: 0.3489 - val_accuracy: 0.9095
Epoch 4/10
3000/3000 [==============================] - 2s 683us/step - loss: 0.3458 - accuracy: 0.9097 - val_loss: 0.3321 - val_accuracy: 0.9112
Epoch 5/10
3000/3000 [==============================] - 2s 675us/step - loss: 0.3198 - accuracy: 0.9133 - val_loss: 0.3194 - val_accuracy: 0.9146
Epoch 6/10
3000/3000 [==============================] - 2s 691us/step - loss: 0.3054 - accuracy: 0.9153 - val_loss: 0.2987 - val_accuracy: 0.9191
Epoch 7/10
3000/3000 [==============================] - 2s 683us/step - loss: 0.2955 - accuracy: 0.9180 - val_loss: 0.2957 -

In [11]:
# predecimos con el modelo grande
print('y_true:', y_true)
y_pred = big_model.predict(X_validation[:10])
print('y_pred:', np.argmax(y_pred, axis=-1))

y_true: [7 2 1 0 4 1 4 9 5 9]
y_pred: [7 2 1 0 4 1 4 9 6 9]


Hemos reducido un 45% el error cometido. ¿?

### Problema sintético
Ahora vamos a trabajar con un problema sintético de regresión. Para ello lo primero es generar un dataset. En este caso solo generaremos un dataset de entrenamiento.

In [12]:
# Datos sintéticos
X_data = np.random.random(size=(1000,2))*5
y_data = (X_data[:,0].astype('int32') + X_data[:,1])/20

Veamos que forma tiene el dataset. ¿Sencillo?

In [15]:
# representamos el dataset
%matplotlib qt
# %matplotlib inline

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(X_data[:,0], X_data[:,1], y_data, c='r')
#plt.show()

ImportError: DLL load failed while importing QtWidgets: No hay suficientes recursos de memoria disponibles para procesar este comando.

![](images/dataset_sintetico.png)

Vemos que es un problema de regresión no continua.
Generemos un modelo relativamente grande para ver si podemos aproximar dichos puntos con una red neuronal.

In [16]:
# capas del modelo
input = Input(shape=2)
layer_1 = layers.Dense(units=5000, activation='sigmoid')(input)
layer_1 = layers.Dense(units=1000, activation='sigmoid')(layer_1)
layer_1 = layers.Dense(units=200, activation='sigmoid')(layer_1)
output = layers.Dense(units=1, activation=None)(layer_1)

# creamos y compilamos el modelo
simple_model = Model(inputs=input, outputs=output)
loss = MSE
metrics = ['mse']
simple_model.compile(optimizer='SGD', loss=loss, metrics=metrics)

# entrenamos
history = simple_model.fit(x=X_data, y=y_data, batch_size=100, epochs=2000)

Epoch 1/2000
10/10 [==============================] - 0s 19ms/step - loss: 1.7201 - mse: 1.7201
Epoch 2/2000
10/10 [==============================] - 0s 19ms/step - loss: 0.1296 - mse: 0.1296
Epoch 3/2000
10/10 [==============================] - 0s 20ms/step - loss: 0.0115 - mse: 0.0115
Epoch 4/2000
10/10 [==============================] - 0s 19ms/step - loss: 0.0115 - mse: 0.0115
Epoch 5/2000
10/10 [==============================] - 0s 19ms/step - loss: 0.0117 - mse: 0.0117
Epoch 6/2000
10/10 [==============================] - 0s 18ms/step - loss: 0.0107 - mse: 0.0107
Epoch 7/2000
10/10 [==============================] - 0s 20ms/step - loss: 0.0107 - mse: 0.0107
Epoch 8/2000
10/10 [==============================] - 0s 20ms/step - loss: 0.0120 - mse: 0.0120
Epoch 9/2000
10/10 [==============================] - 0s 20ms/step - loss: 0.0106 - mse: 0.0106
Epoch 10/2000
10/10 [==============================] - 0s 20ms/step - loss: 0.0107 - mse: 0.0107
Epoch 11/2000
10/10 [==================

10/10 [==============================] - 0s 25ms/step - loss: 0.0081 - mse: 0.0081
Epoch 169/2000
10/10 [==============================] - 0s 23ms/step - loss: 0.0083 - mse: 0.0083
Epoch 170/2000
10/10 [==============================] - 0s 24ms/step - loss: 0.0081 - mse: 0.0081
Epoch 171/2000
10/10 [==============================] - 0s 24ms/step - loss: 0.0078 - mse: 0.0078
Epoch 172/2000
10/10 [==============================] - 0s 24ms/step - loss: 0.0080 - mse: 0.0080
Epoch 173/2000
10/10 [==============================] - 0s 24ms/step - loss: 0.0083 - mse: 0.0083
Epoch 174/2000
10/10 [==============================] - 0s 24ms/step - loss: 0.0080 - mse: 0.0080
Epoch 175/2000
10/10 [==============================] - 0s 24ms/step - loss: 0.0079 - mse: 0.0079
Epoch 176/2000
10/10 [==============================] - 0s 24ms/step - loss: 0.0079 - mse: 0.0079
Epoch 177/2000
10/10 [==============================] - 0s 24ms/step - loss: 0.0080 - mse: 0.0080
Epoch 178/2000
10/10 [=============

10/10 [==============================] - 0s 29ms/step - loss: 0.0050 - mse: 0.0050
Epoch 335/2000
10/10 [==============================] - 0s 28ms/step - loss: 0.0062 - mse: 0.0062
Epoch 336/2000
10/10 [==============================] - 0s 25ms/step - loss: 0.0049 - mse: 0.0049
Epoch 337/2000
10/10 [==============================] - 0s 27ms/step - loss: 0.0051 - mse: 0.0051
Epoch 338/2000
10/10 [==============================] - 0s 27ms/step - loss: 0.0049 - mse: 0.0049
Epoch 339/2000
10/10 [==============================] - 0s 27ms/step - loss: 0.0049 - mse: 0.0049
Epoch 340/2000
10/10 [==============================] - 0s 27ms/step - loss: 0.0050 - mse: 0.0050
Epoch 341/2000
10/10 [==============================] - 0s 26ms/step - loss: 0.0073 - mse: 0.0073
Epoch 342/2000
10/10 [==============================] - 0s 27ms/step - loss: 0.0064 - mse: 0.0064
Epoch 343/2000
10/10 [==============================] - 0s 27ms/step - loss: 0.0048 - mse: 0.0048
Epoch 344/2000
10/10 [=============

10/10 [==============================] - 0s 25ms/step - loss: 0.0032 - mse: 0.0032
Epoch 501/2000
10/10 [==============================] - 0s 25ms/step - loss: 0.0035 - mse: 0.0035
Epoch 502/2000
10/10 [==============================] - 0s 25ms/step - loss: 0.0026 - mse: 0.0026
Epoch 503/2000
10/10 [==============================] - 0s 25ms/step - loss: 0.0026 - mse: 0.0026
Epoch 504/2000
10/10 [==============================] - 0s 25ms/step - loss: 0.0029 - mse: 0.0029
Epoch 505/2000
10/10 [==============================] - 0s 25ms/step - loss: 0.0036 - mse: 0.0036
Epoch 506/2000
10/10 [==============================] - 0s 25ms/step - loss: 0.0033 - mse: 0.0033
Epoch 507/2000
10/10 [==============================] - 0s 26ms/step - loss: 0.0028 - mse: 0.0028
Epoch 508/2000
10/10 [==============================] - 0s 25ms/step - loss: 0.0041 - mse: 0.0041
Epoch 509/2000
10/10 [==============================] - 0s 26ms/step - loss: 0.0045 - mse: 0.0045
Epoch 510/2000
10/10 [=============

10/10 [==============================] - 0s 27ms/step - loss: 0.0052 - mse: 0.0052
Epoch 667/2000
10/10 [==============================] - 0s 27ms/step - loss: 0.0022 - mse: 0.0022
Epoch 668/2000
10/10 [==============================] - 0s 29ms/step - loss: 0.0018 - mse: 0.0018
Epoch 669/2000
10/10 [==============================] - 0s 28ms/step - loss: 0.0019 - mse: 0.0019
Epoch 670/2000
10/10 [==============================] - 0s 28ms/step - loss: 0.0017 - mse: 0.0017
Epoch 671/2000
10/10 [==============================] - 0s 29ms/step - loss: 0.0018 - mse: 0.0018
Epoch 672/2000
10/10 [==============================] - 0s 26ms/step - loss: 0.0021 - mse: 0.0021
Epoch 673/2000
10/10 [==============================] - 0s 29ms/step - loss: 0.0014 - mse: 0.0014
Epoch 674/2000
10/10 [==============================] - 0s 27ms/step - loss: 0.0014 - mse: 0.0014
Epoch 675/2000
10/10 [==============================] - 0s 27ms/step - loss: 0.0015 - mse: 0.0015
Epoch 676/2000
10/10 [=============

...

10/10 [==============================] - 0s 29ms/step - loss: 5.6245e-04 - mse: 5.6245e-04
Epoch 1304/2000
10/10 [==============================] - 0s 30ms/step - loss: 3.0668e-04 - mse: 3.0668e-04
Epoch 1305/2000
10/10 [==============================] - 0s 31ms/step - loss: 2.3014e-04 - mse: 2.3014e-04
Epoch 1306/2000
10/10 [==============================] - 0s 29ms/step - loss: 2.3666e-04 - mse: 2.3666e-04
Epoch 1307/2000
10/10 [==============================] - 0s 29ms/step - loss: 2.5547e-04 - mse: 2.5547e-04
Epoch 1308/2000
10/10 [==============================] - 0s 29ms/step - loss: 2.2157e-04 - mse: 2.2157e-04
Epoch 1309/2000
10/10 [==============================] - 0s 29ms/step - loss: 3.7054e-04 - mse: 3.7054e-04
Epoch 1310/2000
10/10 [==============================] - 0s 29ms/step - loss: 3.3996e-04 - mse: 3.3996e-04
Epoch 1311/2000
10/10 [==============================] - 0s 29ms/step - loss: 2.5211e-04 - mse: 2.5211e-04
Epoch 1312/2000
10/10 [==============================

10/10 [==============================] - 0s 30ms/step - loss: 0.0014 - mse: 0.0014
Epoch 1458/2000
10/10 [==============================] - 0s 31ms/step - loss: 0.0014 - mse: 0.0014
Epoch 1459/2000
10/10 [==============================] - 0s 29ms/step - loss: 0.0020 - mse: 0.0020
Epoch 1460/2000
10/10 [==============================] - 0s 29ms/step - loss: 0.0021 - mse: 0.0021
Epoch 1461/2000
10/10 [==============================] - 0s 29ms/step - loss: 0.0021 - mse: 0.0021
Epoch 1462/2000
10/10 [==============================] - 0s 29ms/step - loss: 0.0022 - mse: 0.0022
Epoch 1463/2000
10/10 [==============================] - 0s 33ms/step - loss: 0.0014 - mse: 0.0014
Epoch 1464/2000
10/10 [==============================] - 0s 34ms/step - loss: 0.0013 - mse: 0.0013
Epoch 1465/2000
10/10 [==============================] - 0s 32ms/step - loss: 9.0934e-04 - mse: 9.0934e-04
Epoch 1466/2000
10/10 [==============================] - 0s 30ms/step - loss: 6.5030e-04 - mse: 6.5030e-04
Epoch 1467

10/10 [==============================] - 0s 28ms/step - loss: 2.2077e-04 - mse: 2.2077e-04
Epoch 1612/2000
10/10 [==============================] - 0s 31ms/step - loss: 3.0187e-04 - mse: 3.0187e-04
Epoch 1613/2000
10/10 [==============================] - 0s 31ms/step - loss: 6.8545e-04 - mse: 6.8545e-04
Epoch 1614/2000
10/10 [==============================] - 0s 29ms/step - loss: 6.8738e-04 - mse: 6.8738e-04
Epoch 1615/2000
10/10 [==============================] - 0s 31ms/step - loss: 6.6113e-04 - mse: 6.6113e-04
Epoch 1616/2000
10/10 [==============================] - 0s 31ms/step - loss: 7.6086e-04 - mse: 7.6086e-04
Epoch 1617/2000
10/10 [==============================] - 0s 29ms/step - loss: 5.7190e-04 - mse: 5.7190e-04
Epoch 1618/2000
10/10 [==============================] - 0s 30ms/step - loss: 6.9701e-04 - mse: 6.9701e-04
Epoch 1619/2000
10/10 [==============================] - 0s 31ms/step - loss: 7.5798e-04 - mse: 7.5798e-04
Epoch 1620/2000
10/10 [==============================

10/10 [==============================] - 0s 29ms/step - loss: 5.0155e-04 - mse: 5.0155e-04
Epoch 1764/2000
10/10 [==============================] - 0s 28ms/step - loss: 4.1005e-04 - mse: 4.1005e-04
Epoch 1765/2000
10/10 [==============================] - 0s 29ms/step - loss: 8.7281e-04 - mse: 8.7281e-04
Epoch 1766/2000
10/10 [==============================] - 0s 29ms/step - loss: 0.0014 - mse: 0.0014
Epoch 1767/2000
10/10 [==============================] - 0s 28ms/step - loss: 0.0016 - mse: 0.0016
Epoch 1768/2000
10/10 [==============================] - 0s 28ms/step - loss: 9.9549e-04 - mse: 9.9549e-04
Epoch 1769/2000
10/10 [==============================] - 0s 29ms/step - loss: 5.1504e-04 - mse: 5.1504e-04
Epoch 1770/2000
10/10 [==============================] - 0s 29ms/step - loss: 4.2179e-04 - mse: 4.2179e-04
Epoch 1771/2000
10/10 [==============================] - 0s 28ms/step - loss: 3.6256e-04 - mse: 3.6256e-04
Epoch 1772/2000
10/10 [==============================] - 0s 29ms/step

10/10 [==============================] - 0s 29ms/step - loss: 2.8696e-04 - mse: 2.8696e-04
Epoch 1917/2000
10/10 [==============================] - 0s 29ms/step - loss: 2.0576e-04 - mse: 2.0576e-04
Epoch 1918/2000
10/10 [==============================] - 0s 28ms/step - loss: 2.5267e-04 - mse: 2.5267e-04
Epoch 1919/2000
10/10 [==============================] - 0s 28ms/step - loss: 2.7308e-04 - mse: 2.7308e-04
Epoch 1920/2000
10/10 [==============================] - 0s 28ms/step - loss: 2.5041e-04 - mse: 2.5041e-04
Epoch 1921/2000
10/10 [==============================] - 0s 29ms/step - loss: 2.2846e-04 - mse: 2.2846e-04
Epoch 1922/2000
10/10 [==============================] - 0s 33ms/step - loss: 2.3416e-04 - mse: 2.3416e-04
Epoch 1923/2000
10/10 [==============================] - 0s 32ms/step - loss: 2.1614e-04 - mse: 2.1614e-04
Epoch 1924/2000
10/10 [==============================] - 0s 32ms/step - loss: 2.1954e-04 - mse: 2.1954e-04
Epoch 1925/2000
10/10 [==============================

Incluso con un modelo grande no hemos podido aproximar el train dataset!!!!
![](images/predicciones_dataset_sintetico1.png)

Si probamos con otro optimizador y cambiando las funciones de activación, incluso con una red mas pequeña conseguimos aproximar el dataset

In [17]:
# capas del modelo
input = Input(shape=2)
layer_1 = layers.Dense(units=500, activation='relu')(input)
layer_1 = layers.Dense(units=200, activation='relu')(layer_1)
layer_1 = layers.Dense(units=100, activation='relu')(layer_1)
output = layers.Dense(units=1, activation=None)(layer_1)

# creamos y compilamos el modelo
simple_model = Model(inputs=input, outputs=output)
loss = MSE
metrics = ['mse']

# compilamos con Adam
simple_model.compile(optimizer='Adam', loss=loss, metrics=metrics)

# entrenamos
history = simple_model.fit(x=X_data, y=y_data, batch_size=100, epochs=2000)

Epoch 1/2000
10/10 [==============================] - 0s 2ms/step - loss: 0.0275 - mse: 0.0275
Epoch 2/2000
10/10 [==============================] - 0s 2ms/step - loss: 0.0023 - mse: 0.0023
Epoch 3/2000
10/10 [==============================] - 0s 2ms/step - loss: 8.1212e-04 - mse: 8.1212e-04
Epoch 4/2000
10/10 [==============================] - 0s 2ms/step - loss: 4.3342e-04 - mse: 4.3342e-04
Epoch 5/2000
10/10 [==============================] - 0s 2ms/step - loss: 2.9591e-04 - mse: 2.9591e-04
Epoch 6/2000
10/10 [==============================] - 0s 2ms/step - loss: 2.5729e-04 - mse: 2.5729e-04
Epoch 7/2000
10/10 [==============================] - 0s 2ms/step - loss: 2.4044e-04 - mse: 2.4044e-04
Epoch 8/2000
10/10 [==============================] - 0s 2ms/step - loss: 2.1274e-04 - mse: 2.1274e-04
Epoch 9/2000
10/10 [==============================] - 0s 2ms/step - loss: 2.0269e-04 - mse: 2.0269e-04
Epoch 10/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.9858e-04 - m

10/10 [==============================] - 0s 2ms/step - loss: 1.8983e-04 - mse: 1.8983e-04
Epoch 81/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.9569e-04 - mse: 1.9569e-04
Epoch 82/2000
10/10 [==============================] - 0s 2ms/step - loss: 2.2832e-04 - mse: 2.2832e-04
Epoch 83/2000
10/10 [==============================] - 0s 2ms/step - loss: 2.0850e-04 - mse: 2.0850e-04
Epoch 84/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.9086e-04 - mse: 1.9086e-04
Epoch 85/2000
10/10 [==============================] - 0s 2ms/step - loss: 2.0876e-04 - mse: 2.0876e-04
Epoch 86/2000
10/10 [==============================] - 0s 2ms/step - loss: 2.1205e-04 - mse: 2.1205e-04
Epoch 87/2000
10/10 [==============================] - 0s 2ms/step - loss: 2.0598e-04 - mse: 2.0598e-04
Epoch 88/2000
10/10 [==============================] - 0s 2ms/step - loss: 2.0809e-04 - mse: 2.0809e-04
Epoch 89/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.9

10/10 [==============================] - 0s 2ms/step - loss: 1.7846e-04 - mse: 1.7846e-04
Epoch 237/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.6731e-04 - mse: 1.6731e-04
Epoch 238/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.6980e-04 - mse: 1.6980e-04
Epoch 239/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.6971e-04 - mse: 1.6971e-04
Epoch 240/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.6836e-04 - mse: 1.6836e-04
Epoch 241/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.7713e-04 - mse: 1.7713e-04
Epoch 242/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.7182e-04 - mse: 1.7182e-04
Epoch 243/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.7612e-04 - mse: 1.7612e-04
Epoch 244/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.6952e-04 - mse: 1.6952e-04
Epoch 245/2000
10/10 [==============================] - 0s 2ms/step - 

10/10 [==============================] - 0s 2ms/step - loss: 1.6620e-04 - mse: 1.6620e-04
Epoch 393/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.4806e-04 - mse: 1.4806e-04
Epoch 394/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.5226e-04 - mse: 1.5226e-04
Epoch 395/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.5639e-04 - mse: 1.5639e-04
Epoch 396/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.4896e-04 - mse: 1.4896e-04
Epoch 397/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.4829e-04 - mse: 1.4829e-04
Epoch 398/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.5376e-04 - mse: 1.5376e-04
Epoch 399/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.6212e-04 - mse: 1.6212e-04
Epoch 400/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.5800e-04 - mse: 1.5800e-04
Epoch 401/2000
10/10 [==============================] - 0s 2ms/step - 

10/10 [==============================] - 0s 2ms/step - loss: 1.3287e-04 - mse: 1.3287e-04
Epoch 548/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.3043e-04 - mse: 1.3043e-04
Epoch 549/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.3358e-04 - mse: 1.3358e-04
Epoch 550/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.4647e-04 - mse: 1.4647e-04
Epoch 551/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.4079e-04 - mse: 1.4079e-04
Epoch 552/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.8007e-04 - mse: 1.8007e-04
Epoch 553/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.7290e-04 - mse: 1.7290e-04
Epoch 554/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.5995e-04 - mse: 1.5995e-04
Epoch 555/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.6097e-04 - mse: 1.6097e-04
Epoch 556/2000
10/10 [==============================] - 0s 2ms/step - 

...

10/10 [==============================] - 0s 2ms/step - loss: 2.7298e-05 - mse: 2.7298e-05
Epoch 1323/2000
10/10 [==============================] - 0s 2ms/step - loss: 2.1877e-05 - mse: 2.1877e-05
Epoch 1324/2000
10/10 [==============================] - 0s 2ms/step - loss: 2.1629e-05 - mse: 2.1629e-05
Epoch 1325/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.9968e-05 - mse: 1.9968e-05
Epoch 1326/2000
10/10 [==============================] - 0s 2ms/step - loss: 2.9824e-05 - mse: 2.9824e-05
Epoch 1327/2000
10/10 [==============================] - 0s 2ms/step - loss: 3.8821e-05 - mse: 3.8821e-05
Epoch 1328/2000
10/10 [==============================] - 0s 2ms/step - loss: 3.1729e-05 - mse: 3.1729e-05
Epoch 1329/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.9414e-05 - mse: 1.9414e-05
Epoch 1330/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.3457e-05 - mse: 1.3457e-05
Epoch 1331/2000
10/10 [==============================] - 0s 2m

10/10 [==============================] - 0s 2ms/step - loss: 1.8983e-05 - mse: 1.8983e-05
Epoch 1477/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.5708e-05 - mse: 1.5708e-05
Epoch 1478/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.3100e-05 - mse: 1.3100e-05
Epoch 1479/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.2543e-05 - mse: 1.2543e-05
Epoch 1480/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.1800e-05 - mse: 1.1800e-05
Epoch 1481/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.1861e-05 - mse: 1.1861e-05
Epoch 1482/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.6111e-05 - mse: 1.6111e-05
Epoch 1483/2000
10/10 [==============================] - 0s 2ms/step - loss: 2.1701e-05 - mse: 2.1701e-05
Epoch 1484/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.6452e-05 - mse: 1.6452e-05
Epoch 1485/2000
10/10 [==============================] - 0s 2m

10/10 [==============================] - 0s 3ms/step - loss: 2.2216e-05 - mse: 2.2216e-05
Epoch 1631/2000
10/10 [==============================] - 0s 2ms/step - loss: 4.5355e-05 - mse: 4.5355e-05
Epoch 1632/2000
10/10 [==============================] - 0s 2ms/step - loss: 4.5897e-05 - mse: 4.5897e-05
Epoch 1633/2000
10/10 [==============================] - 0s 2ms/step - loss: 3.5981e-05 - mse: 3.5981e-05
Epoch 1634/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.9765e-05 - mse: 1.9765e-05
Epoch 1635/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.3097e-05 - mse: 1.3097e-05
Epoch 1636/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.9491e-05 - mse: 1.9491e-05
Epoch 1637/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.2479e-05 - mse: 1.2479e-05
Epoch 1638/2000
10/10 [==============================] - 0s 2ms/step - loss: 8.1203e-06 - mse: 8.1203e-06
Epoch 1639/2000
10/10 [==============================] - 0s 2m

10/10 [==============================] - 0s 2ms/step - loss: 8.7004e-06 - mse: 8.7004e-06
Epoch 1785/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.4212e-05 - mse: 1.4212e-05
Epoch 1786/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.8294e-05 - mse: 1.8294e-05
Epoch 1787/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.6886e-05 - mse: 1.6886e-05
Epoch 1788/2000
10/10 [==============================] - 0s 2ms/step - loss: 2.4009e-05 - mse: 2.4009e-05
Epoch 1789/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.7442e-05 - mse: 1.7442e-05
Epoch 1790/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.3439e-05 - mse: 1.3439e-05
Epoch 1791/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.5963e-05 - mse: 1.5963e-05
Epoch 1792/2000
10/10 [==============================] - 0s 2ms/step - loss: 1.3118e-05 - mse: 1.3118e-05
Epoch 1793/2000
10/10 [==============================] - 0s 2m

10/10 [==============================] - 0s 2ms/step - loss: 8.6772e-06 - mse: 8.6772e-06
Epoch 1939/2000
10/10 [==============================] - 0s 2ms/step - loss: 7.4530e-06 - mse: 7.4530e-06
Epoch 1940/2000
10/10 [==============================] - 0s 2ms/step - loss: 5.5642e-06 - mse: 5.5642e-06
Epoch 1941/2000
10/10 [==============================] - 0s 2ms/step - loss: 7.4430e-06 - mse: 7.4430e-06
Epoch 1942/2000
10/10 [==============================] - 0s 2ms/step - loss: 6.6665e-06 - mse: 6.6665e-06
Epoch 1943/2000
10/10 [==============================] - 0s 2ms/step - loss: 7.1085e-06 - mse: 7.1085e-06
Epoch 1944/2000
10/10 [==============================] - 0s 2ms/step - loss: 7.3609e-06 - mse: 7.3609e-06
Epoch 1945/2000
10/10 [==============================] - 0s 2ms/step - loss: 8.7573e-06 - mse: 8.7573e-06
Epoch 1946/2000
10/10 [==============================] - 0s 2ms/step - loss: 6.1573e-06 - mse: 6.1573e-06
Epoch 1947/2000
10/10 [==============================] - 0s 2m

In [18]:
y_prediction = simple_model.predict(X_data)
%matplotlib qt
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(X_data[:,0], X_data[:,1], y_prediction, c='r')
#plt.show()

ImportError: DLL load failed while importing QtWidgets: No hay suficientes recursos de memoria disponibles para procesar este comando.

![](images/predicciones_dataset_sintetico2.png)
Es un problema muy complejo!

### Modelos no secuenciales
Podemos crear estructuras más complejas, como por ejemplo una red con dos flujos en paralelo.

In [ ]:
# capas del modelo
input = Input(shape=2)
layer_1_a = layers.Dense(units=50, activation='sigmoid')(input)
layer_2_a = layers.Dense(units=10, activation='sigmoid')(layer_1_a)
layer_1_b = layers.Dense(units=50, activation='sigmoid')(input)
layer_2_b = layers.Dense(units=10, activation='sigmoid')(layer_1_b)
layer_2 = tf.concat([layer_2_a, layer_2_b], axis=-1)
output = layers.Dense(units=1, activation=None)(layer_2)
doble_model = Model(inputs=input, outputs=output)
print(doble_model.summary())

### CIFAR 10
Ahora vamos a probar con un dataset mas complejo

In [ ]:
from tensorflow.keras.datasets import cifar10

(X_train, y_train), (X_validation, y_validation) = cifar10.load_data()
X_train = X_train/255
X_validation = X_validation/255

In [ ]:
labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

print('Número de elementos en el dataset = ',X_train.shape[0]+X_validation.shape[0])
plt.title(labels[int(y_train[62])])
plt.imshow(X_train[62])
plt.show()

In [ ]:
# capas de la red
input = Input(shape=(32,32,3))
input_flat = layers.Flatten()(input)
layer = layers.Dense(units=1000, activation='sigmoid')(input_flat)
layer = layers.Dense(units=500, activation='sigmoid')(layer)
layer = layers.Dense(units=200, activation='sigmoid')(layer)
output = layers.Dense(units=10, activation='softmax')(layer)

# creamos el modelo
model = Model(inputs=input, outputs=output)
print(model.summary())

# optimizador
optimizer = SGD(learning_rate=0.01)

# función loss
loss = SparseCategoricalCrossentropy(from_logits=False)

# métrica
metrics = ['accuracy']

# compilamos el modelo
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
history = model.fit(x=X_train, y=y_train, batch_size=30, epochs=20,
                           validation_data=(X_validation, y_validation))

In [ ]:
from aux_func import show_history
show_history(history)

Probar con pocas capas y neuronas e ir aumentando para ver la evolución del score. (fijar número de capas y hacer una gráfica nº neuronas vs score)
¿Qué pasa si la profundidad de la red es muy grande? Intentar dar alguna explicación de porque pasa eso.

El score es malo por varias razones: el tipo de arquitectura, la función de activación y el optimizador no son los correctos.

Si por ejemplo probamos con un random forest obtenemos lo siguiente:

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=10)
clf.fit(X_train.reshape((X_train.shape[0],-1)), y_train.reshape((-1,1)))
print('score rf = ',clf.score(X_validation.reshape((X_validation.shape[0],-1)), y_validation.reshape((-1,1))))

Si cambiamos la función de activación...

In [ ]:
# capas de la red
input = Input(shape=(32,32,3))
input_flat = layers.Flatten()(input)
layer = layers.Dense(units=2000, activation='relu')(input_flat)
layer = layers.Dense(units=1000, activation='relu')(layer)
layer = layers.Dense(units=500, activation='relu')(layer)
layer = layers.Dense(units=200, activation='relu')(layer)
layer = layers.Dense(units=100, activation='relu')(layer)
output = layers.Dense(units=10, activation='softmax')(layer)

# creamos el modelo
model = Model(inputs=input, outputs=output)
print(model.summary())

# optimizador
optimizer = SGD(learning_rate=0.01)

# función loss
loss = SparseCategoricalCrossentropy(from_logits=False)

# métrica
metrics = ['accuracy']

# compilamos el modelo
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

history = model.fit(x=X_train, y=y_train, batch_size=30, epochs=20,
                    validation_data=(X_validation, y_validation))

### CIFAR 100
Intentar conseguir el mejor score (fine tuning) con el dataset CIFAR 100 (se puede utilizar arquitecturas no secuenciales).
Hacer distintas predicciones con la red y plotear los resultados.

Comparar, con la función show_history, el performance de varias (<10) NN's con distintas neuronas y capas.
Obtener las siguientes gráficas (longitud <20):
- score vs Batch_size (con la arquitectura conseguida previamente)
- score vs nº neuronas (con las capas de la arquitectura conseguida previamente)

Hacer la comparativa de scores (acc, recall, precision, f1, confusion matrix) entre los siguientes modelos:
- Random Forest
- Otro modelo a elegir de Machine Learning
- Una red neuronal
- Random forest sobre el histograma del LBP
- Una red neuronal sobre el histograma del LBP

LBP: https://www.pyimagesearch.com/2015/12/07/local-binary-patterns-with-python-opencv/